* Dates:
    * Older moves might not be torrented
    * Month: Blockbusters are released in May and December, No good movies released January - April, Kids are on summer cvaction during summer months.
* Production Budget: 
    * Might be outliers. Take log if outliers, Remove outliers, Create new feature that is binning production budget
* Rating: Is there a relationship with Number of torrents, look at mean, median torrents per rating. Maybe group ratings together for example G, PG, PG-13 in one group R in another
* Genre as a feature. Maybe keep a subset maybe create new ratings by combing other ratings

In [1]:
import pandas as pd
import numpy as np

import operator
import string
from collections import defaultdict

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns

import plotly
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.tools import FigureFactory as FF
plotly.tools.set_credentials_file(username='bbiggs', api_key='jrgs2lfv3u')

# Read in data and convert dates to datetime objects

In [2]:
# csv into pandas dataframe
df = pd.read_csv('data/data.csv', encoding='latin-1')

# update dates to datetime objects
df['Released'] = pd.to_datetime(df['Released'])
df['Year'] = pd.DatetimeIndex(df['Released']).year
df['Year_Int'] = pd.to_numeric(df['Year'])
df['Month'] = pd.DatetimeIndex(df['Released']).month

In [3]:
# dictionary of year counts
yr_dict = df['Year'].value_counts().to_dict()

# sort dictionary by year
yr_lst = sorted(yr_dict.items(), key=operator.itemgetter(0))
yr_lst = yr_lst[::-1]

years = [year for year, count in yr_lst]
year_count = [count for year, count in yr_lst]

trace = go.Bar(
    x=years,
    y=year_count,
    marker=dict(
        color='rgb(26, 118, 255)'
    ))

data = [trace]
layout = go.Layout(
    title='Movies Released per Year',
    xaxis=dict(
        title='Release Year',
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    yaxis=dict(
        title='Number of Movies',
        titlefont=dict(
            size=16,
            color='rgb(107, 107, 107)'
        ),
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='style-bar')

# Trim dataframe by years that are relevant

In [4]:
# trim dataframe to "relevant" years
before = len(df)
yr_cut_bot = 1995
yr_cut_top = 2015
mask = (df['Year'] >= yr_cut_bot) & (df['Year'] <= yr_cut_top)
df = df.loc[mask]
df.sort_values('Year').head()
after = len(df)

print('{0} entries lost ({1}%) due to date cutoff between {2} and {3}'.format(before-after, 
                                                round((before/after)/before *100, 2), yr_cut_bot, yr_cut_top))

267 entries lost (0.07%) due to date cutoff between 1995 and 2015


In [5]:
# dictionary of year counts
yr_dict = df['Year'].value_counts().to_dict()

# sort dictionary by year
yr_lst = sorted(yr_dict.items(), key=operator.itemgetter(0))
yr_lst = yr_lst[::-1]

years = [year for year, count in yr_lst]
year_count = [count for year, count in yr_lst]

trace = go.Bar(
    x=years,
    y=year_count,
    marker=dict(
        color='rgb(26, 118, 255)'
    ))

data = [trace]
layout = go.Layout(
    title='Movies Released per Year ({0}-{1})'.format(yr_cut_bot, yr_cut_top),
    xaxis=dict(
        title='Release Year',
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    yaxis=dict(
        title='Number of Movies',
        titlefont=dict(
            size=16,
            color='rgb(107, 107, 107)'
        ),
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='style-bar')

In [6]:
'''
# 1998
mask =  (df_yr['Year'] == 1998) & (df_yr['Total_Torrents'] > 100)
df_yr = df_yr[~mask]

# 1999
mask =  (df_yr['Year'] == 1999) & (df_yr['Total_Torrents'] > 200)
df_yr = df_yr[~mask]

# 2000
mask =  (df_yr['Year'] == 2000) & (df_yr['Total_Torrents'] > 200)
df_yr = df_yr[~mask]

# 2001
mask =  (df_yr['Year'] == 2001) & (df_yr['Total_Torrents'] > 300)
df_yr = df_yr[~mask]

# 2002
mask =  (df_yr['Year'] == 2002) & (df_yr['Total_Torrents'] > 300)
df_yr = df_yr[~mask]

# 2003
mask =  (df_yr['Year'] == 2003) & (df_yr['Total_Torrents'] > 300)
df_yr = df_yr[~mask]

# 2004
# 2005
mask =  (df_yr['Year'] == 2005) & (df_yr['Total_Torrents'] > 300)
df_yr = df_yr[~mask]

# 2006
mask =  (df_yr['Year'] == 2006) & (df_yr['Total_Torrents'] > 300)
df_yr = df_yr[~mask]

# 2007
mask =  (df_yr['Year'] == 2007) & (df_yr['Total_Torrents'] > 450)
df_yr = df_yr[~mask]

# 2010
mask =  (df_yr['Year'] == 2010) & (df_yr['Total_Torrents'] > 800)
df_yr = df_yr[~mask]

# 2014
mask =  (df_yr['Year'] == 2014) & (df_yr['Total_Torrents'] > 850)
df_yr = df_yr[~mask]
'''

"\n# 1998\nmask =  (df_yr['Year'] == 1998) & (df_yr['Total_Torrents'] > 100)\ndf_yr = df_yr[~mask]\n\n# 1999\nmask =  (df_yr['Year'] == 1999) & (df_yr['Total_Torrents'] > 200)\ndf_yr = df_yr[~mask]\n\n# 2000\nmask =  (df_yr['Year'] == 2000) & (df_yr['Total_Torrents'] > 200)\ndf_yr = df_yr[~mask]\n\n# 2001\nmask =  (df_yr['Year'] == 2001) & (df_yr['Total_Torrents'] > 300)\ndf_yr = df_yr[~mask]\n\n# 2002\nmask =  (df_yr['Year'] == 2002) & (df_yr['Total_Torrents'] > 300)\ndf_yr = df_yr[~mask]\n\n# 2003\nmask =  (df_yr['Year'] == 2003) & (df_yr['Total_Torrents'] > 300)\ndf_yr = df_yr[~mask]\n\n# 2004\n# 2005\nmask =  (df_yr['Year'] == 2005) & (df_yr['Total_Torrents'] > 300)\ndf_yr = df_yr[~mask]\n\n# 2006\nmask =  (df_yr['Year'] == 2006) & (df_yr['Total_Torrents'] > 300)\ndf_yr = df_yr[~mask]\n\n# 2007\nmask =  (df_yr['Year'] == 2007) & (df_yr['Total_Torrents'] > 450)\ndf_yr = df_yr[~mask]\n\n# 2010\nmask =  (df_yr['Year'] == 2010) & (df_yr['Total_Torrents'] > 800)\ndf_yr = df_yr[~mask]\n\

In [7]:
df_a = df[['Prod_Budget', 'Year', 'Month', 'Runtime', 'Total_Torrents']]

In [8]:
#fig = FF.create_scatterplotmatrix(df_a, diag='histogram', index='Prod_Budget', height=1000, width=1000)
#py.iplot(fig, filename='Histograms along Diagonal Subplots')

In [9]:
# set of grouped genres as list of strings
unq_genres = df['Genre'].unique()
unq_genres = unq_genres.tolist()

# set grouped genres as list of lists
lst_grp_genres = []
for lst in unq_genres:
    temp = []
    for genre in lst.split(','):
         temp.append(genre)
    lst_grp_genres.append(temp)
    
# set list of individual genres
ind_genre = set()

for lst in unq_genres:
    for genre in lst.split(','):
        ind_genre.add(genre.strip())
ind_genre = sorted(ind_genre)

In [10]:
# dictionary of genre counts
count = defaultdict(lambda:0)
for genre in ind_genre:
    count[genre] = df['Genre'].str.contains(genre).sum()

# sort genre dictionary by year
genres_srt_lst = sorted(count.items(), key=operator.itemgetter(1))
genres_srt_lst = genres_srt_lst[::-1]

# make genre lists
genre = [genre for genre, count in genres_srt_lst]
genre_count = [count for genre, count in genres_srt_lst]

In [11]:
trace = go.Bar(
    x=genre,
    y=genre_count,
    marker=dict(
        color='rgb(26, 118, 255)'
    ))

data = [trace]
layout = go.Layout(
    title='Genre Occurences ({0}-{1})'.format(yr_cut_bot, yr_cut_top),
    xaxis=dict(
        title='Genre',
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    yaxis=dict(
        title='Occurences in Classification',
        titlefont=dict(
            size=16,
            color='rgb(107, 107, 107)'
        ),
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='style-bar')

In [12]:
# split genre strings into a numpy array
def split_to_array(ser):
    split_array = np.array(ser.strip().replace(',','').split(' '))
    return pd.Series(split_array)

# turn numpy array into count of genre occurances
genres = df['Genre'].apply(split_to_array)
genres = pd.Series(genres.values.ravel()).dropna()
genres = genres.value_counts().sort_values(ascending=False)

# return genre that occurs the most (in total data set) in the list of genres (for that row)
def convert_frequency(ser, genres=genres):
    split_array = np.array(ser.strip().replace(',','').split(' '))
    genre = genres.loc[split_array].argmax()
    return genre

df['Genre_Single'] = df['Genre'].apply(convert_frequency)

In [13]:
# dictionary of genre counts
year_genre_dict = defaultdict(lambda:0)
genre_ct_dict = defaultdict(lambda:0)

# sorted list of years from dataframe
years = sorted(df['Year'].unique().tolist())

# build dictionary of genre occurences by genre for each year
for year in years:
    df_temp = df[df['Year'] == year]

    for g in genre:
        genre_ct_dict[g] = df_temp['Genre'].str.contains(g).sum()
    
    year_genre_dict[year] = genre_ct_dict

In [15]:
genre_dict = defaultdict(lambda:0)
year_ct_dict = defaultdict(lambda:0)

# sorted list of years from dataframe
years = sorted(df['Year_Int'].unique().tolist())

# make dictionary of year, count by genre
for gen in genre:
    df_temp = df[df['Genre'].str.contains(gen)]

    genre_dict[gen] = df_temp['Year'].value_counts().to_dict()

In [16]:
def make_trace(x_years, y_counts, name_genre):
    trace = go.Bar(
        x = x_years, 
        y = y_counts, 
        name = name_genre)
    return trace

In [17]:
traces = []

for genre, info in genre_dict.items():
    year_lst = []
    count_lst = []
    
    for year, count in info.items():
        year_lst.append(year)
        count_lst.append(count)
        
    traces.append(make_trace(year_lst, count_lst, genre))
    
data = traces
layout = go.Layout(
    barmode='stack'
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='stacked-bar')

# Select only significant values from dataframe
# df_yr => df_sub

In [18]:
# select only genres of significance
genre = ['Action', 'Adventure', 'Comedy', 'Crime', 'Drama']
df_sub = df_yr.loc[df_yr['Genre_Single'].isin(genre)]

# select only genres of significance
ratings = ['PG-13', 'PG', 'G', 'R']
df_sub = df_sub.loc[df_sub['Rated'].isin(ratings)]

#df_sub['Runtime'].value_counts()
#df_sub['Genre_Single'].value_counts()
#df_sub['Rated'].value_counts()
df_sub.describe()

NameError: name 'df_yr' is not defined

In [ ]:
# entire dataframe
plt.rcParams['figure.figsize'] = (15, 15)
_ = pd.tools.plotting.scatter_matrix(df_sub)

In [ ]:
import plotly.plotly as py
from plotly.tools import FigureFactory as FF

df_a = df_yr[['Prod_Budget', 'Year', 'Month', 'Total_Torrents']]
fig = FF.create_scatterplotmatrix(df_a, diag='histogram', index='Month',
                                  height=800, width=800)
py.iplot(fig, filename='Histograms along Diagonal Subplots')

In [ ]:
from patsy import dmatrices
patsy_formula = 'Total_Torrents ~ Prod_Budget + Year + Month + Runtime + Genre_Single'
y, x = dmatrices(patsy_formula, data=df_sub, return_type='dataframe')

In [ ]:
import statsmodels.api as sm
model = sm.OLS(y, x)
results = model.fit()
results.summary()

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(x, y)
mod_lr_score = model.score(x, y)
mod_lr_coef = model.coef_
model.results

In [ ]:
from sklearn import cross_validation as cv
from sklearn import metrics

x_train, x_test, y_train, y_test = cv.train_test_split(x,y,test_size=0.20,random_state=1234)

model = LinearRegression().fit(x_train, y_train)

# store results
mean_sq_err = metrics.mean_squared_error(y_train,model.predict(x_train))
cv_mod_score = model.score(x_train, y_train)

In [ ]:
# reset x, y otherwise errors occur
y, x = dmatrices(patsy_formula, data=df_sub, return_type='dataframe')

from sklearn.cross_validation import KFold
kf = KFold(len(df_sub), n_folds=10, shuffle=True)

for train_index, test_index in kf:
    x_train, x_test = x.iloc[train_index], x.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    clf2 = LinearRegression().fit(x.iloc[train_index], y.iloc[train_index])

# store results
mean_sq_errKf = metrics.mean_squared_error(y_train,model.predict(x_train))
cvKf_mod_score = clf2.score(x,y)

In [ ]:
#NORMAL RESULTS
print('Model Linear Regression Score = {0}'.format(mod_lr_score))
print('            Mean Square Error = {0}'.format(mean_sq_err))
print(' Cross Validation Model Score = {0}'.format(cv_mod_score))
print('     Mean Squred Error K-Fold = {0}'.format(mean_sq_errKf))
print('Cross Val. K-Fold Model Score = {0}'.format(cvKf_mod_score))

In [ ]:
fig = plt.figure(figsize=(12,8))
fig = sm.graphics.plot_regress_exog(results,'Prod_Budget', fig=fig)

---
# Log Transform

In [ ]:
df.columns

In [ ]:
df_sub['log_budg']=np.log(df_sub.Prod_Budget)
#df_sub['log_year']=np.log(df_sub.Year)
#df_sub['log_run']=np.log(df_sub.Runtime)
df_sub['log_tor']=np.log(df_sub.Total_Torrents)

trans = df_sub[['log_budg', 'Year', 'log_tor']]
plt.rcParams['figure.figsize'] = (15, 15)
pd.tools.plotting.scatter_matrix(trans)

In [ ]:
log_patsy_formula = 'log_tor ~ log_budg + Year + Month'
y, x = dmatrices(log_patsy_formula, data=df_sub, return_type='dataframe')

In [ ]:
import plotly.plotly as py
from plotly.tools import FigureFactory as FF

df_a = df_sub[['log_budg', 'Year', 'Month', 'log_tor']]
fig = FF.create_scatterplotmatrix(df_a, diag='histogram', index='Month',
                                  height=800, width=800)
py.iplot(fig, filename='Histograms along Diagonal Subplots')

In [ ]:
import statsmodels.formula.api as smf
results = smf.ols(formula=log_patsy_formula, data=df_sub,).fit()
results.summary()

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(x, y)

# store results
log_mod_lr_score = model.score(x,y)

In [ ]:
from sklearn import cross_validation as cv
from sklearn import metrics

x_train, x_test, y_train, y_test = cv.train_test_split(x,y,test_size=0.20,random_state=1234)

model = LinearRegression().fit(x_train, y_train)

# store results
log_mean_sq_err = metrics.mean_squared_error(y_train,model.predict(x_train))
log_cv_mod_score = model.score(x_train, y_train)

In [ ]:
# reset x, y otherwise errors occur
y, x = dmatrices(log_patsy_formula, data=df_sub, return_type='dataframe')

from sklearn.cross_validation import KFold
kf = KFold(len(df_sub), n_folds=10, shuffle=True)

for train_index, test_index in kf:
    x_train, x_test = x.iloc[train_index], x.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    clf2 = LinearRegression().fit(x.iloc[train_index], y.iloc[train_index])

# store results
log_mean_sq_errKf = metrics.mean_squared_error(y_train,model.predict(x_train))
log_cvKf_mod_score = clf2.score(x,y)

In [ ]:
#LOG RESULTS
print('Log Model Linear Regression Score = {0}'.format(log_mod_lr_score))
print('            Log Mean Square Error = {0}'.format(log_mean_sq_err))
print(' Log Cross Validation Model Score = {0}'.format(log_cv_mod_score))
print('     Log Mean Squred Error K-Fold = {0}'.format(log_mean_sq_errKf))
print('Log Cross Val. K-Fold Model Score = {0}'.format(log_cvKf_mod_score))

In [ ]:
df_TEST = pd.read_csv('data/test_data2.csv', encoding='latin-1')

df_TEST['log_budg']=np.log(df_TEST.Prod_Budget)
df_TEST['log_run']=np.log(df_TEST.Runtime)
df_TEST['log_tor']=np.log(df_TEST.Total_Torrents)

def split_to_array(ser):
    split_array = np.array(ser.strip().replace(',','').split(' '))
    return pd.Series(split_array)

genres = df_yr.Genre.apply(split_to_array)
genres = pd.Series(genres.values.ravel()).dropna()
genres = genres.value_counts().sort_values(ascending=False)

def convert_frequency(ser, genres=genres):
    split_array = np.array(ser.strip().replace(',','').split(' '))
    genre = genres.loc[split_array].argmax()
    return genre

df_TEST['Genre_Single'] = df_TEST.Genre.apply(convert_frequency)

log_patsy_formula_test = 'log_tor ~ log_budg + Year + Month + log_run + Genre_Single'
y, x = dmatrices(log_patsy_formula_test, data=df_TEST, return_type='dataframe')

print(clf2.score(x_test, y_test))
print(metrics.mean_squared_error(y_test,model.predict(x_test)))

In [ ]:
_ = plt.plot(y, model.predict(x), 'bo')

In [ ]:
plt.figure(figsize=(25,10))

ind = np.arange(len(yr_dict))
width = 0.35

bar_year = [year for year, count in yr_lst]
bar_count = [count for year, count in yr_lst]

plt.bar(ind, bar_count, width, color='r')

plt.ylabel('Count')
plt.xlabel('Year')
plt.title('Number of Torrents per Year')
plt.xticks(ind + width/2., (bar_year), rotation='vertical')
plt.yticks(np.arange(0, 91, 5))

plt.show()

In [ ]:
#log_tor ~ log_budg + Year + Month + log_run + Genre_Single'

In [ ]:
fig = plt.figure(figsize=(12,8))
fig = sm.graphics.plot_regress_exog(results,'log_budg', fig=fig)

In [ ]:
fig = plt.figure(figsize=(12,8))
fig = sm.graphics.plot_regress_exog(results,'Year', fig=fig)

In [ ]:
fig = plt.figure(figsize=(12,8))
fig = sm.graphics.plot_regress_exog(results,'Month', fig=fig)